## Tool Node 사용 방법

먼저 tool 세팅

In [3]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional

class GoogleNews:
    """
    구글 뉴스를 검색하고 결과를 반환하는 클래스입니다.
    """

    def __init__(self):
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:

        if not news_list:
            print("해당 키워드의 뉴스가 없습니다.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=ko&gl=KR&ceid=KR:ko"
        else:
            url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

In [4]:
from dotenv import load_dotenv

from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from typing import List, Dict

# API 키 정보 로드
load_dotenv()

# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


@tool
def python_code_interpreter(code: str):
    """Call to execute python code."""
    return PythonAstREPLTool().invoke(code)

from langgraph.prebuilt import ToolNode, tools_condition

# 도구 리스트 생성
tools = [search_news, python_code_interpreter]

# ToolNode 초기화
tool_node = ToolNode(tools)



Tool Node 수동 호출은 다음과 같은 포멧이어야 함.

In [5]:
# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
# AIMessage 객체이어야 함
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_news",  # 도구 이름
            "args": {"query": "AI"},  # 도구 인자
            "id": "tool_call_id",  # 도구 호출 ID
            "type": "tool_call",  # 도구 호출 유형
        }
    ],
)

# 도구 노드를 통한 메시지 처리 및 날씨 정보 요청 실행
tool_node.invoke({"messages": [message_with_single_tool_call]})


{'messages': [ToolMessage(content='[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE5xUnhGTHN1TDhxTENWajh5Rk52dk1yUlhaU0RIYmFsQjA4Ukc1VGRKbHNMVkM2TloteUdYdkN6UnRFd3NWaUpScVJtQjRVNFBoSkVNd2tHejJqUGNSZDYxWWprM1hJalppLUE?oc=5", "content": "AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레"}, {"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1DSndQNnVQRk85N3VacVhjazRfaDhMM0dFZ0k1ZXU0b1F1WGZoQTJ0M1lQbFAyYTVWRXRsVlZlTmo2b3BXYkEzWlJaTkVmXy1RM2FPMGpSdjBnUUtKenk0RFJoWThUeWlMUFE?oc=5", "content": "\\"GPT 성능 향상 속도 둔화\\"...오픈AI, \'오라이온\' 개선 위해 전략 수정 - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiiAFBVV95cUxOcngtSVhkeDhWSW1xbnZhVkowR200MDB5MjVzQkxybUhhSGJDbjJyMkVzMFg0X21QRVNscVpBazJuNDRNYlg2SUdGYk1UeDFCNnlZSlBLTjd6WGVHcGdhdjRZcXgwNXFFTUlBay1hRDJjRVRISDFWR1ZOWDRRQm56S3NXQ01mTTRB0gGcAUFVX3lxTE1UdExXTzU1NUxiSmZEZlFScGZkN1dvQUVZTmVXLVFxc2xMMUs2M0xjZ1lDdkFGMHNwakR3MUp0akJQLTZZa0twekFoTjgwNkplWkU0NUlYVTZYcG5kLVRQYktBTWZ5TWlNNHVtREJKU1BKLXlEZmtrUlV2cGlrMTY3R2pwRC04NVBjUFhQZnQ3

In [6]:
# 다중 도구 호출을 포함하는 AI 메시지 객체 생성 및 초기화
message_with_multiple_tool_calls = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "search_news",
            "args": {"query": "AI"},
            "id": "tool_call_id",
            "type": "tool_call",
        },
        {
            "name": "python_code_interpreter",
            "args": {"code": "print(1+2+3+4)"},
            "id": "tool_call_id",
            "type": "tool_call",
        },
    ],
)

# 생성된 메시지를 도구 노드에 전달하여 다중 도구 호출 실행
tool_node.invoke({"messages": [message_with_multiple_tool_calls]})


{'messages': [ToolMessage(content='[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE5xUnhGTHN1TDhxTENWajh5Rk52dk1yUlhaU0RIYmFsQjA4Ukc1VGRKbHNMVkM2TloteUdYdkN6UnRFd3NWaUpScVJtQjRVNFBoSkVNd2tHejJqUGNSZDYxWWprM1hJalppLUE?oc=5", "content": "AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레"}, {"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1DSndQNnVQRk85N3VacVhjazRfaDhMM0dFZ0k1ZXU0b1F1WGZoQTJ0M1lQbFAyYTVWRXRsVlZlTmo2b3BXYkEzWlJaTkVmXy1RM2FPMGpSdjBnUUtKenk0RFJoWThUeWlMUFE?oc=5", "content": "\\"GPT 성능 향상 속도 둔화\\"...오픈AI, \'오라이온\' 개선 위해 전략 수정 - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiiAFBVV95cUxOcngtSVhkeDhWSW1xbnZhVkowR200MDB5MjVzQkxybUhhSGJDbjJyMkVzMFg0X21QRVNscVpBazJuNDRNYlg2SUdGYk1UeDFCNnlZSlBLTjd6WGVHcGdhdjRZcXgwNXFFTUlBay1hRDJjRVRISDFWR1ZOWDRRQm56S3NXQ01mTTRB0gGcAUFVX3lxTE1UdExXTzU1NUxiSmZEZlFScGZkN1dvQUVZTmVXLVFxc2xMMUs2M0xjZ1lDdkFGMHNwakR3MUp0akJQLTZZa0twekFoTjgwNkplWkU0NUlYVTZYcG5kLVRQYktBTWZ5TWlNNHVtREJKU1BKLXlEZmtrUlV2cGlrMTY3R2pwRC04NVBjUFhQZnQ3

In [7]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 및 도구 바인딩
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

도구 호출 확인

In [8]:
# 도구 호출 확인
model_with_tools.invoke("처음 5개의 소수를 출력하는 python code 를 작성해줘").tool_calls

[{'name': 'python_code_interpreter',
  'args': {'code': 'def first_n_primes(n):\n    primes = []\n    num = 2  # Starting from the first prime number\n    while len(primes) < n:\n        is_prime = True\n        for i in range(2, int(num**0.5) + 1):\n            if num % i == 0:\n                is_prime = False\n                break\n        if is_prime:\n            primes.append(num)\n        num += 1\n    return primes\n\n# Get the first 5 prime numbers\nfirst_n_primes(5)'},
  'id': 'call_hsF2W7kZJmblIaFUawzqF0bL',
  'type': 'tool_call'}]

tool_bind 한 모델은 tool_calls가 자동 내포하고 있기 때문에 바로 tool_node로 가도 상관없다.

In [9]:
tool_node.invoke(
    {
        "messages": [
            model_with_tools.invoke(
                "처음 5개의 소수를 출력하는 python code 를 작성해줘"
            )
        ]
    }
)

{'messages': [ToolMessage(content='[2, 3, 5, 7, 11]', name='python_code_interpreter', tool_call_id='call_UQuhzXRog6EBnzRMs5C08aHj')]}

## 최종 완성 코드

In [10]:
# LangGraph 워크플로우 상태 및 메시지 처리를 위한 타입 임포트
from langgraph.graph import StateGraph, MessagesState, START, END


# LLM 모델을 사용하여 메시지 처리 및 응답 생성, 도구 호출이 포함된 응답 반환
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


# 메시지 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(MessagesState)

# 에이전트와 도구 노드 정의 및 워크플로우 그래프에 추가
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 워크플로우 시작점에서 에이전트 노드로 연결
workflow.add_edge(START, "agent")

# 에이전트 노드에서 조건부 분기 설정, 도구 노드 또는 종료 지점으로 연결
workflow.add_conditional_edges("agent", tools_condition)

# 도구 노드에서 에이전트 노드로 순환 연결
workflow.add_edge("tools", "agent")

# 에이전트 노드에서 종료 지점으로 연결
workflow.add_edge("agent", END)


# 정의된 워크플로우 그래프 컴파일 및 실행 가능한 애플리케이션 생성
app = workflow.compile()


In [11]:
# 검색 질문 수행
for chunk in app.stream(
    {"messages": [("human", "search google news about AI")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

search google news about AI
================================== Ai Message ==================================
Tool Calls:
  search_news (call_ltg2DtSBVQL46zTVELEx56cM)
 Call ID: call_ltg2DtSBVQL46zTVELEx56cM
  Args:
    query: AI
================================= Tool Message =================================
Name: search_news

[{"url": "https://news.google.com/rss/articles/CBMiakFVX3lxTE1DSndQNnVQRk85N3VacVhjazRfaDhMM0dFZ0k1ZXU0b1F1WGZoQTJ0M1lQbFAyYTVWRXRsVlZlTmo2b3BXYkEzWlJaTkVmXy1RM2FPMGpSdjBnUUtKenk0RFJoWThUeWlMUFE?oc=5", "content": "\"GPT 성능 향상 속도 둔화\"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스"}, {"url": "https://news.google.com/rss/articles/CBMiVkFVX3lxTE5xWkdpNHlQUEthaldYQnMzaVluTVota0thOHFRbDA3VW5UZmtXOS0wRF9QM2RmZFprSkdyTVdPb012cWlMdEszS0YxcW1vbWFqa25UR2t3?oc=5", "content": "\"AI 성장 한계 직면\"…오픈AI, 기술 돌파구 마련 위해 新 전략 가동하나 - ZD넷 코리아"}, {"url": "https://news.google.com/rss/articles/CBMic0FVX3lxTFBJTDJ0bEJ0NWV